In [1]:
# One factor behind deep learning’s success is the availability of a wide range of layers that can be composed in creative ways to design 
# architectures suitable for a wide variety of tasks. For instance, researchers have invented layers specifically for handling images, 
# text, looping over sequential data, and performing dynamic programming. Sooner or later, 
# you will need a layer that does not exist yet in the deep learning framework. 
# In these cases, you must build a custom layer. In this section, we show you how.
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [2]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [3]:
layer = CenteredLayer()
layer(torch.tensor([1.0, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [4]:
net = nn.Sequential(nn.LazyLinear(128), CenteredLayer())

/data2/miniconda3/envs/d2l-pytorch/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [5]:
# As an extra sanity check, we can send random data through the network and check that the mean is in fact 0. 
# Because we are dealing with floating point numbers, we may still see a very small nonzero number due to quantization.
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-3.7253e-09, grad_fn=<MeanBackward0>)

In [6]:
# Now that we know how to define simple layers, let’s move on to defining layers with parameters 
# that can be adjusted through training. We can use built-in functions to create parameters, 
# which provide some basic housekeeping functionality. In particular, they govern access, initialization, sharing, saving, 
# and loading model parameters. This way, among other benefits, we will not need to write custom serialization routines for every custom layer.

# Now let’s implement our own version of the fully connected layer. Recall that this layer requires two parameters, 
# one to represent the weight and the other for the bias. In this implementation, we bake in the ReLU activation as a default. 
# This layer requires two input arguments: in_units and units, which denote the number of inputs and outputs, respectively.

class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [7]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.6041,  0.5023, -0.3819],
        [ 0.5514,  1.2492,  1.0090],
        [ 1.1140,  0.1160,  0.7600],
        [-0.6272, -0.8034,  0.1312],
        [-0.2197,  0.8615,  2.3766]], requires_grad=True)

In [8]:
# We can directly carry out forward propagation calculations using custom layers.
linear(torch.rand(2, 5))

tensor([[1.3520, 2.0628, 1.8716],
        [0.5713, 0.7158, 0.0000]])

In [9]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

1. Design a layer that takes an input and computes a tensor reduction:

To design a layer that computes the tensor reduction, you can create a custom PyTorch layer that inherits from `nn.Module` and implements the `forward` method. In this case, we'll compute the sum of all elements in the input tensor.

```python
import torch
import torch.nn as nn

class TensorReduction(nn.Module):
    def forward(self, x):
        return torch.sum(x)

# Example usage
input_data = torch.randn(3, 4)
reduction_layer = TensorReduction()
output = reduction_layer(input_data)
print(output)
```

2. Design a layer that returns the leading half of the Fourier coefficients of the data:

To design a layer that returns the leading half of the Fourier coefficients of the data, you can create a custom PyTorch layer that inherits from `nn.Module` and implements the `forward` method. In this case, we'll use the `torch.fft.fft` function to compute the Fourier coefficients and then return the leading half.

```python
import torch
import torch.nn as nn

class LeadingHalfFourier(nn.Module):
    def forward(self, x):
        # Compute the Fourier coefficients
        fourier_coefficients = torch.fft.fft(x)

        # Get the leading half of the Fourier coefficients
        n = fourier_coefficients.shape[-1]
        leading_half = fourier_coefficients[..., :n // 2]

        return leading_half

# Example usage
input_data = torch.randn(3, 4)
fourier_layer = LeadingHalfFourier()
output = fourier_layer(input_data)
print(output)
```

In this example, the `LeadingHalfFourier` layer computes the Fourier coefficients of the input data using `torch.fft.fft` and then returns the leading half of the coefficients. Note that this example assumes the input data is a 1D signal. If you have multi-dimensional data (e.g., images), you can use `torch.fft.fftn` or `torch.fft.fft2` to compute the Fourier coefficients accordingly.

傅里叶系数（Fourier coefficients）是指在傅里叶分析中，将一个信号（如时间域信号）分解为正弦和余弦函数的系数。傅里叶分析的主要目的是将信号从时域转换到频域，以便更容易地分析信号中的频率成分。在频域中，信号表示为一系列正弦和余弦波的叠加，每个波的振幅和相位由傅里叶系数给出。

对于一个离散信号x(t)，通过使用离散傅里叶变换（Discrete Fourier Transform, DFT），可以计算其傅里叶系数。离散傅里叶变换将时域信号转换为频域信号，其中每个频率成分的振幅和相位由傅里叶系数表示。计算傅里叶系数的公式如下：

X(k) = Σ [x(t) * exp(-j * 2π * k * t / N)]

其中，X(k)是傅里叶系数，x(t)是时域信号，N是信号的长度，k是频率成分的索引，j是虚数单位。

在深度学习和信号处理中，傅里叶系数可以帮助我们分析和处理信号中的频率成分。例如，在图像处理中，我们可以使用傅里叶变换来分析图像中的高频和低频成分，然后应用滤波器来消除噪声或增强某些特征。在语音处理中，傅里叶变换可以帮助我们识别语音信号中的基频和谐波成分，从而提高语音识别和合成的性能。